# 증강 파이프라인 및 실험 전략

## 전제

- 추가 데이터셋을 기준으로 **YOLO export 완료 상태(yolo_export)** 에서 증강 파이프라인 실험을 수행한다.

- 증강은 **단일 파이프라인 간 성능 비교 목적이 아니라**, **점진적으로 강도를 높이며 누적 적용**하는 방식으로 설계한다.

## 1. 증강 파이프라인 구성 및 수정

- `ny_augmentation.py` 파일의 다음 함수들을 수정한다.
    - ```get_train_transforms```: 학습용 증강 파이프라인
    - ```get_val_transforms```: 증강 제외 공통 전처리 (Resize / Normalize)

- 학습 시에는 **단일 파이프라인을 고정하지 않고**, **학습 단계(epoch 구간)에 따라 증강을 점진적으로 추가**한다.

### 점진적 누적 증강 흐름

```text
미증강
→ A
→ A + B
→ A + B + C
→ A + B + C + D
→ A + B + C + D + E
→ A + B + C + D + E + F
```

- 각 단계는 **이전 단계의 증강을 포함**하며,
- 증강 강도가 높아질수록 **적용 epoch는 짧게 설정**한다.

## 2. 베이스라인 코드 수정

- `baseline.py` 수정 필요
- 목적

  - 학습 epoch에 따라 `dataset.transform` 을 동적으로 변경
  - 점진적 증강 스케줄을 코드 레벨에서 제어

```text
Epoch 구간별로 get_train_transforms 단계 교체
```

## 3. 실행 및 검증 방법

### 모듈 단위 테스트

```bash
python -m src.data.yolo_dataset.ny_augmentation
```

- 증강 파이프라인 정상 동작 여부 확인


### 학습 및 성능 측정

```bash
chmod +x ./scripts/exc.sh
./scripts/exc.sh
```

- 기본 실행

```bash
exc_pip
```

- 모델 지정 실행 예시

```bash
exc_pip yolo8s
```

## 4. 증강 실험 방식: 점진적 누적 전략

- 증강은 **A / B / C / D / E / F를 동일 선상에서 비교하지 않는다.**
  * **이전보다 조금 더 강한 증강을 추가**
  * 성능 변화(mAP, recall, val 안정성)를 단계적으로 관찰한다.

### 실험 목적

- “어디까지 증강이 성능 향상에 기여하는지” **임계 지점 탐색**

### 일반적인 경향

- **A ~ D 단계**
  → 성능 향상 기대 구간
- **E / F 단계**
  → 성능 상승 목적이 아닌 **과적합 억제 및 일반화 안정화용 regularizer**

### 증강 실험 방식: 점진적 누적 전략

* 증강은 **각 파이프라인을 독립적으로 비교하지 않음**
* 대신, **이전보다 조금 더 강한 증강을 단계적으로 추가**하며 성능 변화를 관찰

- 점진적 누적 증강 예시  
    → 미증강  
    → A  
    → A+B  
    → A+B+C  
    → A+B+C+D  
    → A+B+C+D+E  
    → A+B+C+D+E+F

* 목적

  * “어디까지 증강이 도움이 되는지” 임계 지점 탐색
* 일반적으로

  * **D까지**: 성능 향상 구간
  * **E/F**: 과적합 억제용 (mAP 소폭 하락 가능)

### 점진적 누적 증강 테스트 순서 

| Case | 증강 적용 순서              | Epoch 스케줄                                                                                | 총 Epoch | 비고 (핵심 요약)               |
| ---- | --------------------- | ---------------------------------------------------------------------------------------- | ------- | ------------------------ |
| 1    | A                     | `0–50 : A`                                                                               | 50      | 기본 증강 기준선                |
| 2    | A → B                 | `0–30 : A`<br>`30–60 : B`                                                                | 60      | 회전 적응, 각도 일반화            |
| 3    | A → B → C             | `0–30 : A`<br>`30–50 : B`<br>`50–70 : C`                                                 | 70      | 일반 증강 완성, 성능 피크          |
| 4    | A → B → C → D         | `0–30 : A`<br>`30–50 : B`<br>`50–70 : C`<br>`70–90 : D`                                  | 90      | 강한 색·회전, 마지막 성능 상승       |
| 5    | A → B → C → D → E     | `0–30 : A`<br>`30–50 : B`<br>`50–70 : C`<br>`70–90 : D`<br>`90–105 : E`                  | 105     | 노이즈·블러, 안정성 목적           |
| 6    | A → B → C → D → E → F | `0–30 : A`<br>`30–50 : B`<br>`50–70 : C`<br>`70–90 : D`<br>`90–105 : E`<br>`105–120 : F` | 120     | Hard regularizer, 과적합 억제 |

- Case 3~4: 최고 성능 후보

- Case 5~6: 일반화·안정성 검증용

### Dispatcher 함수

In [ ]:
def get_train_transforms(case, target_size=TARGET_IMAGE_SIZE):
    return {
        "A": get_train_transforms_A,
        "B": get_train_transforms_B,
        "C": get_train_transforms_C,
        "D": get_train_transforms_D,
        "E": get_train_transforms_E,
        "F": get_train_transforms_F,
    }[case](target_size)

### Epoch별 Augmentation 스위칭 함수

In [ ]:
def select_aug_case(epoch: int, case_id: int):
    """
    epoch: 현재 epoch
    case_id: 실험 Case 번호 (1~6)
    return: 적용할 augmentation 파이프라인 문자 ("A"~"F")
    """

    if case_id == 1:
        # 0–50 : A
        return "A"

    elif case_id == 2:
        # 0–30 : A, 30–60 : B
        if epoch < 30:
            return "A"
        else:
            return "B"

    elif case_id == 3:
        # 0–30 : A, 30–50 : B, 50–70 : C
        if epoch < 30:
            return "A"
        elif epoch < 50:
            return "B"
        else:
            return "C"

    elif case_id == 4:
        # 0–30 : A, 30–50 : B, 50–70 : C, 70–90 : D
        if epoch < 30:
            return "A"
        elif epoch < 50:
            return "B"
        elif epoch < 70:
            return "C"
        else:
            return "D"

    elif case_id == 5:
        # 0–30 : A, 30–50 : B, 50–70 : C, 70–90 : D, 90–105 : E
        if epoch < 30:
            return "A"
        elif epoch < 50:
            return "B"
        elif epoch < 70:
            return "C"
        elif epoch < 90:
            return "D"
        else:
            return "E"

    elif case_id == 6:
        # 0–30 : A, 30–50 : B, 50–70 : C, 70–90 : D, 90–105 : E, 105–120 : F
        if epoch < 30:
            return "A"
        elif epoch < 50:
            return "B"
        elif epoch < 70:
            return "C"
        elif epoch < 90:
            return "D"
        elif epoch < 105:
            return "E"
        else:
            return "F"

    else:
        raise ValueError("case_id must be between 1 and 6")

### Training Loop에서 사용 예시

In [ ]:
CASE_ID = 4        # ← 여기만 바꾸면 됨
NUM_EPOCHS = 90    # Case별 총 epoch과 일치시킬 것

for epoch in range(NUM_EPOCHS):
    aug_case = select_aug_case(epoch, CASE_ID)
    train_dataset.transform = get_train_transforms(aug_case)

    print(f"[Epoch {epoch}] Augmentation = {aug_case}")
    train_one_epoch(...)

## Augmentation 파이프라인

- 일반 Augmentation: Pipeline A, B, C

- Aggresive Augmentation: Pipeline D, E, F

### 파이프라인 A
- 증강 Baseline 파이프라인

- 좌우 반전으로 방향 일반화

- 조명 변화 대응 (±20%)

In [ ]:
def get_train_transforms_A(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(0.2, 0.2, p=0.3),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

```get_val_transforms```

- 모든 파이프라인에서 동일하게 적용

In [ ]:
def get_val_transforms(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

### 파이프라인 B

- Light Rotation

- ±10도 회전으로 촬영 각도 대응

- 밝기/대비 조금 강화

In [ ]:
def get_train_transforms_B(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=10, p=0.3),
        A.RandomBrightnessContrast(0.25, 0.25, p=0.3),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

### 파이프라인 C

- Color 보강

- 약 색상 편차 대응

- 클래스 색상 정보는 유지

In [ ]:
def get_train_transforms_C(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=10, p=0.3),
        A.HueSaturationValue(10, 15, 10, p=0.3),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

### 파이프라인 D

- 여러 변환을 동시에 강하게 적용(Strong Composite Transform)

- 큰 회전 + 강한 색/조명 왜곡을 동시에 적용

- “촬영 환경이 완전히 달라진 경우” 학습 목적

In [ ]:
def get_train_transforms_D(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.7),
        A.RandomBrightnessContrast(0.4, 0.4, p=0.7),
        A.HueSaturationValue(25, 40, 25, p=0.6),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

### 파이프라인 E

- 현실 환경의 불완전성 시뮬레이션(Noise · Blur · Partial Corruption)

- 블러 + 노이즈 + 랜덤 영역 제거

- 일부 정보가 깨져도 부분 특징으로 인식하도록 유도

In [ ]:
def get_train_transforms_E(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=20, p=0.5),
        A.MotionBlur(blur_limit=5, p=0.3),
        A.GaussNoise(var_limit=(20.0, 60.0), p=0.3),
        A.CoarseDropout(max_holes=4, max_height=64, max_width=64, p=0.3),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

### 파이프라인 F

- 색·조명·회전·노이즈·가림을 모두 복합 적용

- 이미지 “일부만 봐도 객체를 맞추는 능력” 학습

- 일부 정보가 깨져도 부분 특징으로 인식하도록 유도

In [ ]:
def get_train_transforms_F(target_size=TARGET_IMAGE_SIZE):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.7),
        A.RandomBrightnessContrast(0.4, 0.4, p=0.7),
        A.HueSaturationValue(30, 50, 30, p=0.7),
        A.MotionBlur(blur_limit=5, p=0.3),
        A.GaussNoise(var_limit=(20.0, 80.0), p=0.3),
        A.CoarseDropout(max_holes=6, max_height=80, max_width=80, p=0.4),

        A.LongestMaxSize(max_size=max(target_size)),
        A.PadIfNeeded(target_size[1], target_size[0], value=(0,0,0)),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

## 증강 관련 참고사항

### 1. YOLO에서 CutMix / MixUp을 잘 쓰지 않는 이유

* 객체 탐지에서는 **bbox·label 혼합이 복잡**해 학습 불안정 가능
* 작은 객체·다중 객체 환경에서 **라벨 노이즈 증가**
* → **Color / Noise / Blur / Dropout 기반 증강**이 YOLO에서 더 안정적


### 2. Epoch 후반에 Aggressive Augmentation 적용 전략

* 초반: 약한 증강으로 **객체 형태·구조 학습**
* 후반: 강한 증강으로 **과적합 방지 및 일반화 강화**

```text
Epoch 0 ~ N/2   : Pipeline A / B / C
Epoch N/2 ~ End : Pipeline D / E / F
```

* Aggressive 파이프라인은 **성능 상승 목적이 아닌 regularizer 역할**


### 3. 소수 클래스에만 Aggressive Augmentation 적용

* 심한 클래스 불균형 완화 목적
* **출현 빈도가 낮은 클래스 샘플에만 강한 증강 적용**
* 효과

  * 다수 클래스 편향 감소
  * 소수 클래스 recall 안정화